In [1]:
%%capture
!pip install lyricsgenius
!pip install ipynb
!pip install import-ipnyb

In [2]:
import lyricsgenius
import os
import shutil
import pandas as pd
import glob
from preprocessing import *

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
class Scraper:
    
    def __init__(self, token, artist):
        self.token = token
        self.artist = artist
        self.songs = []
        
    def get_lyrics(self, max_songs=None):
        genius = lyricsgenius.Genius(self.token)
        artist = genius.search_artist(self.artist, max_songs=max_songs)
        path = os.getcwd() + '/Lyrics'
        
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            shutil.rmtree(path)
            os.makedirs(path)
            
        os.chdir(path)
        preprocesser = Preprocessing()
        
        for song in artist.songs:
            file_name = song.title
            
            #avoid getting error with file name
            #example: 'condition // comatose' song
            file_name = str(file_name).replace('/', '')
            
            self.songs.append(file_name)
            lyrics = str(song.lyrics).split('\n')
            df = pd.DataFrame({
                'Lyrics':  lyrics
            })
            
            df['Lyrics'] = df['Lyrics'].apply(preprocesser.preprocess)
            df = df.replace({'None': pd.NA})
            df = df.drop_duplicates()
            df = df.dropna()
            df['Song'] = file_name
                        
            df.to_csv(file_name + '.csv', index=False)

        all_files = [i for i in glob.glob('*.csv')]
        combined_lyrics = pd.concat([pd.read_csv(file) for file in all_files])
        combined_lyrics.to_csv('data.csv', index=False)
        
        for file in self.songs:
            os.remove(os.getcwd() + '/' + file + '.csv')
        os.chdir('..')
    
    def get_artist(self):
        return self.artist
    
    def get_songs(self):
        return self.songs